In [1]:
# -----------------------------------------------------------------------------
# Embedding Vectorization for Justice Dataset
# (Synchronized with Traditional Vectorization)
# -----------------------------------------------------------------------------
# Description:
#   Creates 8 embedding-based vectorization variants (Word2Vec, FastText, GloVe, etc.)
#   for the Justice Dataset. Uses fixed master splits for regression, binary, and multiclass tasks.
#   Structure and outputs are synchronized with traditional_vectorization.ipynb.
# -----------------------------------------------------------------------------

# ---------------------------
# 1. Setup and Imports
# ---------------------------
from generate_master_split import ensure_master_split
ensure_master_split()

import os
import joblib
import pandas as pd

from helpers_text_preprocessing import (
    load_spacy_model,
    enhanced_spacy_preprocessing
)
from helpers_embedding_vectorization import (
    vectorize_embeddings,
    save_results
)

MASTER_SPLIT_PATH = "Artifacts/master_split_indices.joblib"

# ---------------------------
# 2. Load Dataset
# ---------------------------
justice_df = pd.read_csv("../Justice_Dataset/justice.csv")
required_cols = ["facts", "first_party_winner", "issue_area", "term"]
missing = [c for c in required_cols if c not in justice_df.columns]
if missing:
    raise ValueError(f"Missing columns in dataset: {missing}")

print(f"✅ Dataset loaded successfully: {justice_df.shape[0]} rows")

# ---------------------------
# 3. Text Preprocessing
# ---------------------------
nlp = load_spacy_model("en_core_web_sm")
justice_df["facts_clean"] = enhanced_spacy_preprocessing(
    texts=justice_df["facts"],
    nlp=nlp,
    keep_negations=True,
    preserve_legal_terms=True,
    batch_size=100,
    n_process=4
)
print("Text preprocessing complete.")

# ---------------------------
# 4. Embedding Variants
# ---------------------------
embedding_variants = [
    "word2vec_avg", "word2vec_tfidf",
    "fasttext_avg", "fasttext_tfidf",
    "glove_avg", "glove_tfidf",
    "trainable_fasttext_avg", "trainable_fasttext_tfidf"
]

# ---------------------------
# 5. Path Configuration
# ---------------------------
try:
    BASE_DIR = os.path.abspath(os.path.dirname(__file__))
except NameError:
    BASE_DIR = os.getcwd()

ARTIFACTS_DIR = os.path.join(BASE_DIR, "Artifacts")
os.makedirs(ARTIFACTS_DIR, exist_ok=True)

for subdir in ["binary", "multiclass", "regression"]:
    os.makedirs(os.path.join(ARTIFACTS_DIR, subdir), exist_ok=True)

print(f"Saving all results under: {ARTIFACTS_DIR}")

# ---------------------------
# 6. Load Master Split Indices (3 Tasks)
# ---------------------------
try:
    all_splits = joblib.load(MASTER_SPLIT_PATH)
    print("Loaded fixed master split indices for all tasks.")
except FileNotFoundError:
    raise RuntimeError("Master split not found. Run generate_master_split.py first.")

# ---------------------------
# 7. Binary Classification
# ---------------------------
print("\n Binary classification (first_party_winner)")
binary_df = justice_df[["facts_clean", "first_party_winner"]].dropna()
split_bin = all_splits["binary"]

binary_splits = {
    "X_train": binary_df.loc[split_bin["train_idx"], "facts_clean"],
    "X_val":   binary_df.loc[split_bin["val_idx"], "facts_clean"],
    "X_test":  binary_df.loc[split_bin["test_idx"], "facts_clean"],
    "y_train": binary_df.loc[split_bin["train_idx"], "first_party_winner"],
    "y_val":   binary_df.loc[split_bin["val_idx"], "first_party_winner"],
    "y_test":  binary_df.loc[split_bin["test_idx"], "first_party_winner"],
}

for variant in embedding_variants:
    print(f"→ Vectorizing binary task with {variant} ...")
    results = vectorize_embeddings(binary_splits, variant)
    save_results(
        results,
        os.path.join(ARTIFACTS_DIR, "binary", f"binary_{variant}_300f.joblib"),
        "binary"
    )

# ---------------------------
# 8. Multiclass Classification
# ---------------------------
print("\nMulticlass classification (issue_area)")
multiclass_df = justice_df[["facts_clean", "issue_area"]].dropna()
merge_map = {"Private Action": "Miscellaneous", "Interstate Relations": "Miscellaneous"}
multiclass_df["issue_area"] = multiclass_df["issue_area"].replace(merge_map)

split_multi = all_splits["multiclass"]

multiclass_splits = {
    "X_train": multiclass_df.loc[split_multi["train_idx"], "facts_clean"],
    "X_val":   multiclass_df.loc[split_multi["val_idx"], "facts_clean"],
    "X_test":  multiclass_df.loc[split_multi["test_idx"], "facts_clean"],
    "y_train": multiclass_df.loc[split_multi["train_idx"], "issue_area"],
    "y_val":   multiclass_df.loc[split_multi["val_idx"], "issue_area"],
    "y_test":  multiclass_df.loc[split_multi["test_idx"], "issue_area"],
}

for variant in embedding_variants:
    print(f"→ Vectorizing multiclass task with {variant} ...")
    results = vectorize_embeddings(multiclass_splits, variant)
    save_results(
        results,
        os.path.join(ARTIFACTS_DIR, "multiclass", f"multiclass_{variant}_300f.joblib"),
        "multiclass"
    )

# ---------------------------
# 9. Regression (Term Prediction)
# ---------------------------
print("\nRegression task (term)")
justice_df["term_year"] = pd.to_numeric(justice_df["term"], errors="coerce")
regression_df = justice_df[["facts_clean", "term_year"]].dropna()

split_reg = all_splits["regression"]

regression_splits = {
    "X_train": regression_df.loc[split_reg["train_idx"], "facts_clean"],
    "X_val":   regression_df.loc[split_reg["val_idx"], "facts_clean"],
    "X_test":  regression_df.loc[split_reg["test_idx"], "facts_clean"],
    "y_train": regression_df.loc[split_reg["train_idx"], "term_year"],
    "y_val":   regression_df.loc[split_reg["val_idx"], "term_year"],
    "y_test":  regression_df.loc[split_reg["test_idx"], "term_year"],
}

for variant in embedding_variants:
    print(f"→ Vectorizing regression task with {variant} ...")
    results = vectorize_embeddings(regression_splits, variant)
    save_results(
        results,
        os.path.join(ARTIFACTS_DIR, "regression", f"regression_{variant}_300f.joblib"),
        "regression"
    )

# ---------------------------
# 10. Completion
# ---------------------------
print("\n All embedding variants successfully generated and saved with '_300f.joblib' naming convention.")

Master split already exists at: Artifacts/master_split_indices.joblib
✅ Dataset loaded successfully: 3303 rows


INFO:helpers_text_preprocessing:Successfully loaded spaCy model: en_core_web_sm
INFO:helpers_text_preprocessing:Starting enhanced text preprocessing pipeline...
INFO:helpers_text_preprocessing:  Step 1/4: Unicode normalization and mojibake removal
INFO:helpers_text_preprocessing:  Step 2/4: Preserving legal multiword terms
INFO:helpers_text_preprocessing:  Step 3/4: spaCy processing (batch_size=100, n_process=4)
INFO:helpers_text_preprocessing:  Step 4/4: Final cleanup and validation
INFO:helpers_text_preprocessing:Preprocessing complete. Average tokens per document: 92.1
INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:collected 13053 word types from a corpus of 211815 raw words and 2301 sentences
INFO:gensim.models.word2vec:Creating a fresh vocabulary
INFO:gensim.utils:Word2Vec lifecycle event {'msg': 'effective_min_count=2 retains 8705 unique wor

Text preprocessing complete.
Saving all results under: /Users/fpmuga/Documents/00-MathClasses/2025-2026/1_FirstSemester_2025-2026/MATH_130.1-UV1_2025-1/MUGA_LAB/03_Notebooks/MUGA_LAB/Artifacts
Loaded fixed master split indices for all tasks.

 Binary classification (first_party_winner)
→ Vectorizing binary task with word2vec_avg ...


INFO:gensim.models.word2vec:EPOCH 0: training on 211815 raw words (186431 effective words) took 0.1s, 1979870 effective words/s
INFO:gensim.models.word2vec:EPOCH 1: training on 211815 raw words (186507 effective words) took 0.1s, 2137061 effective words/s
INFO:gensim.models.word2vec:EPOCH 2: training on 211815 raw words (186555 effective words) took 0.1s, 2138094 effective words/s
INFO:gensim.models.word2vec:EPOCH 3: training on 211815 raw words (186458 effective words) took 0.1s, 1975400 effective words/s
INFO:gensim.models.word2vec:EPOCH 4: training on 211815 raw words (186599 effective words) took 0.1s, 1906472 effective words/s
INFO:gensim.utils:Word2Vec lifecycle event {'msg': 'training on 1059075 raw words (932550 effective words) took 0.5s, 1980609 effective words/s', 'datetime': '2025-10-11T16:44:03.450126', 'gensim': '4.3.3', 'python': '3.10.18 | packaged by conda-forge | (main, Jun  4 2025, 14:46:00) [Clang 18.1.8 ]', 'platform': 'macOS-26.0.1-arm64-arm-64bit', 'event': 'trai

Saved word2vec_avg results to Artifacts/binary/binary_word2vec_avg_300f.joblib
→ Vectorizing binary task with word2vec_tfidf ...


...

Saved glove_tfidf results to Artifacts/regression/regression_glove_tfidf_300f.joblib
→ Vectorizing regression task with trainable_fasttext_avg ...


INFO:gensim.models.word2vec:EPOCH 0: training on 210531 raw words (189716 effective words) took 0.1s, 1873418 effective words/s
INFO:gensim.models.word2vec:EPOCH 1: training on 210531 raw words (189851 effective words) took 0.1s, 1977108 effective words/s
INFO:gensim.models.word2vec:EPOCH 2: training on 210531 raw words (189641 effective words) took 0.1s, 1909881 effective words/s
INFO:gensim.models.word2vec:EPOCH 3: training on 210531 raw words (189813 effective words) took 0.1s, 1977222 effective words/s
INFO:gensim.models.word2vec:EPOCH 4: training on 210531 raw words (189799 effective words) took 0.1s, 1964138 effective words/s
INFO:gensim.models.word2vec:EPOCH 5: training on 210531 raw words (189969 effective words) took 0.1s, 1956900 effective words/s
INFO:gensim.models.word2vec:EPOCH 6: training on 210531 raw words (189804 effective words) took 0.1s, 1915710 effective words/s
INFO:gensim.models.word2vec:EPOCH 7: training on 210531 raw words (189842 effective words) took 0.1s, 20

Saved trainable_fasttext_avg results to Artifacts/regression/regression_trainable_fasttext_avg_300f.joblib
→ Vectorizing regression task with trainable_fasttext_tfidf ...


INFO:gensim.utils:FastText lifecycle event {'update': False, 'trim_rule': 'None', 'datetime': '2025-10-11T16:48:15.168006', 'gensim': '4.3.3', 'python': '3.10.18 | packaged by conda-forge | (main, Jun  4 2025, 14:46:00) [Clang 18.1.8 ]', 'platform': 'macOS-26.0.1-arm64-arm-64bit', 'event': 'build_vocab'}
INFO:gensim.utils:FastText lifecycle event {'msg': 'training model with 4 workers on 12996 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=5 shrink_windows=True', 'datetime': '2025-10-11T16:48:15.168479', 'gensim': '4.3.3', 'python': '3.10.18 | packaged by conda-forge | (main, Jun  4 2025, 14:46:00) [Clang 18.1.8 ]', 'platform': 'macOS-26.0.1-arm64-arm-64bit', 'event': 'train'}
INFO:gensim.models.word2vec:EPOCH 0: training on 210531 raw words (189678 effective words) took 0.7s, 276527 effective words/s
INFO:gensim.models.word2vec:EPOCH 1: training on 210531 raw words (189674 effective words) took 0.7s, 265071 effective words/s
INFO:gensim.models.word2vec:EPO

Saved trainable_fasttext_tfidf results to Artifacts/regression/regression_trainable_fasttext_tfidf_300f.joblib

 All embedding variants successfully generated and saved with '_300f.joblib' naming convention.
